# Stock Trading Portfolio App Simulation

## Static Perspective Widget

Load the entire stocks CSV data file and load it into interactive Perspective Viewer

- Data is static via pandas dataframe
- Perspective jupyter widget is loaded

In [1]:
import pandas as pd

# Load the stock CSV data
datafile = r"./data/tech_trades.csv"
df = pd.read_csv(datafile, nrows=500_000)

#### Traditional way using `plotly`

Graphing trades over time using plotly what typical data science users do. Interactive(ish!).

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

# Plotting the wave and amps using Plotly
pio.templates.default = 'plotly_dark'
fig = go.Figure()

# iterate through tickers and graphs each with line charts
tickers = list(df['ticker'].unique())
for ticker in tickers:      
    # Add the wave trace
    xdf = df[df['ticker'] == ticker][['date', 'ticker', 'close']]
    fig.add_trace(go.Scatter(x=xdf['date'], y=xdf['close'], mode='lines', name=ticker))

fig.show()

#### Interactive Perspective View

Slicing and dicing data using _Tableau-like_ perspective viewer.

In [ ]:
from perspective.widget import PerspectiveWidget

perspective_view = PerspectiveWidget(df, theme="Dracula")
# Display the widget in Jupyter Notebook
perspective_view

Adding view pre-configuration. 
- User preferences can be saved/restored from a JSON database.

In [ ]:
import pandas as pd
from perspective.widget import PerspectiveWidget

# Load the stock CSV data
datafile = r"./data/tech_trades.csv"
df = pd.read_csv(datafile, nrows=500_000)

perspective_view = PerspectiveWidget(df, columns=["ticker", "broker", "bid_price", "shares", "trade_value"], theme="Dracula")

# Display the widget in Jupyter Notebook
perspective_view

## Streaming Perspective Viewer

Now, we stream data from the CSV file day-by-day sing the `DemoDataStreamPlayer` helper class. This class simply only update Perspective Viewer with a single day frame of data from the CSV file.

In [ ]:
from stocks_utils import TRADES_SCHEMA, DemoDataStreamPlayer
from perspective.widget import PerspectiveWidget

# stock data file path
datafile = "data/tech_trades.csv"

# Create a PerspectiveWidget
view1 = PerspectiveWidget(
    TRADES_SCHEMA, 
    index="ticker", 
    sort=[["shares", "asc"]], 
    theme="Dracula", 
    group_by=["ticker"],
    aggregates={"broker": "first"},
    )
# load demo data
demo = DemoDataStreamPlayer(
    data_filepath=datafile,
    widgets=view1,
    interval=0.01,
)
# run to play the data frame-by-frame
demo.start()

# display the widget
view1

#### Multiple Refreshing Windows

In [ ]:
from stocks_utils import TRADES_SCHEMA, DemoDataStreamPlayer
from perspective.widget import PerspectiveWidget
from ipywidgets import VBox, HBox


# stock data file path
datafile = "data/tech_trades.csv"

# Create a PerspectiveWidget
v1 = PerspectiveWidget(
    TRADES_SCHEMA, 
    index="ticker",
    plugin="Y Area",
    group_by=["ticker"],
    split_by=["broker"],
    columns=["ticker", "broker", "shares", "trade_value"],
    aggregates={"broker": "last", "shares": "sum", "trade_value": "sum"},
    sort=[["ticker", "asc"]],
    theme="Dracula",
    settings=False,
    )
# Create a PerspectiveWidget
v2 = PerspectiveWidget(
    TRADES_SCHEMA, 
    index="ticker",
    plugin="Heatmap",
    group_by=["ticker"],
    split_by=["broker"],
    columns=["ticker", "broker", "shares", "trade_value"],
    aggregates={"broker": "last", "shares": "sum", "trade_value": "sum"},
    sort=[["ticker", "asc"]],
    theme="Dracula",
    settings=False,
    )
# load demo data
demo = DemoDataStreamPlayer(
    data_filepath=datafile,
    widgets=[v1, v2],
    interval=0.01,
)
# run to play the data frame-by-frame
demo.start()

# display the widget
HBox(children=[v1, v2])